In [4]:
import os
import requests
import pandas as pd

data = pd.read_csv('locations/coordinates_by_city.csv')

api_key = os.getenv("owm_brx")
if not api_key:
    raise ValueError("The API Key for the OpenWeatherMap API is not set in your Environment Variables.")

src = pd.read_csv('locations/coordinates_by_city.csv')
src
# tgt_ls = src['city_name'].values
# responses = []

# for city in tgt_ls:
#     current_weather_url = 'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}'
#     response = requests.get(url=current_weather_url)
#     responses.append(response.json())
# output = pd.DataFrame({'city_name_input':tgt_ls, 'api_json_respnose':responses})
# output.to_csv('geo_api_lat_long_output.csv', index= False )

,city_name_input,latitude,longitude,country_name,iso_2_letter_country_code
0,New York City,40.712728,-74.006015,United States of America (the),US
1,Los Angeles,34.053691,-118.242766,United States of America (the),US
2,Chicago,41.875562,-87.624421,United States of America (the),US
3,Toronto,43.653482,-79.383935,Canada,CA
4,Mexico City,19.432630,-99.133178,Mexico,MX
...,...,...,...,...,...
95,Bogotá,4.653465,-74.083645,Colombia,CO
96,Lima,-12.062107,-77.036526,Peru,PE
97,Santiago,9.869479,-83.798075,Costa Rica,CR
98,Caracas,10.506093,-66.914601,Venezuela (Bolivarian Republic of),VE


In [9]:
import ast
import pandas as pd

data = pd.read_csv('C:/Users/patri/OneDrive/Desktop/work/snowflake_sandbox/models/raw/open_weather_map_api/python/locations/coordinates/geo_api_lat_long_output.csv')
json_ls = [ast.literal_eval(item)[0] if item!= '[]' else None for item in data['api_json_respnose'].values]

lats = [json_item['lat'] if isinstance(json_item, dict) else None for json_item in json_ls]
data['latitude'] = lats

longs = [json_item['lon'] if isinstance(json_item, dict) else None for json_item in json_ls]
data['longitude'] = longs

nations = [json_item['country'] if isinstance(json_item, dict) else None for json_item in json_ls]
data['iso_2_letter_country_code'] = nations

data = pd.merge(data
               ,pd.read_csv('C:/Users/patri/OneDrive/Desktop/work/snowflake_sandbox/models/raw/open_weather_map_api/python/locations/names/iso_country_codes.csv')
               ,left_on='iso_2_letter_country_code'
               ,right_on='iso_2_letter_code'
               )
data = data[['city_name_input', 'latitude', 'longitude', 'country_name', 'iso_2_letter_country_code']]
data = data.rename(columns={ 'city_name_input':'city_name', 'iso_2_letter_country_code':'country_code' })
data.to_csv('C:/Users/patri/OneDrive/Desktop/work/snowflake_sandbox/models/raw/open_weather_map_api/python/locations/coordinates_by_city.csv'
            ,index=False
           )